### Notebook to demonstrate EfficientDet workflow

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

![image](https://developer.nvidia.com/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png)


### The workflow in a nutshell

- Creating a dataset
- Upload kitti dataset to the service
- Running dataset convert
- Getting a PTM from NGC
- Model Actions
    - Train
    - Evaluate
    - Prune, retrain
    - Export
    - Convert
    - Inference on TAO
    - Inference on TRT

### Table of contents

1. [Create datasets ](#head-1)
1. [List the created datasets](#head-2)
1. [Dataset convert Action](#head-3)
1. [Create model ](#head-4)
1. [List models](#head-5)
1. [Assign train, eval datasets](#head-6)
1. [Assign PTM](#head-7)
1. [Actions](#head-8)
1. [Train](#head-9)
1. [Evaluate](#head-10)
1. [Optimize: Apply specs for prune](#head-12)
1. [Optimize: Apply specs for retrain](#head-13)
1. [Optimize: Run actions](#head-14)
1. [Export: FP32](#head-15)
1. [Export: INT8](#head-16)
1. [Model convert using TAO-Converter](#head-17)
1. [TAO inference](#head-18)
1. [TRT inference](#head-19)

### Requirements
Please find the server requirements [here](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_setup.html#)

In [ ]:
import json
import os
import requests
import uuid
import time
from IPython.display import clear_output

### FIXME

1. Assign a workdir in FIXME1
2. Assign the ip_address and port_number in FIXME 2 ([info](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_rest_api.html))
3. Assign the ngc_api_key variable in FIXME 3
4. Set path of dataset folder in FIXME 4
5. Choose between default and custom dataset in FIXME 5

In [ ]:
# Define workspaces and other variables
workdir = "workdir_efficientdet" # FIXME1
host_url = "http://<ip_address>:<port_number>" # FIXME2 example: https://10.137.149.22:32334
# In host machine, node ip_address and port number can be obtained as follows,
# ip_address: hostname -i
# port_number: kubectl get service ingress-nginx-controller -o jsonpath='{.spec.ports[0].nodePort}'
ngc_api_key = "<ngc_api_key>" # FIXME3 example: zZYtczM5amdtdDcwNjk0cnA2bGU2bXQ3bnQ6NmQ4NjNhMDItMTdmZS00Y2QxLWI2ZjktNmE5M2YxZTc0OGyM

In [ ]:
# Exchange NGC_API_KEY for JWT
response = requests.get(f"{host_url}/api/v1/login/{ngc_api_key}")
user_id = response.json()["user_id"]
print("User ID",user_id)
token = response.json()["token"]
print("JWT",token)

# Set base URL
base_url = f"{host_url}/api/v1/user/{user_id}"
print("API Calls will be forwarded to",base_url)

headers = {"Authorization": f"Bearer {token}"}

In [ ]:
# Creating workdir
if not os.path.isdir(workdir):
    os.makedirs(workdir)

### Create datasets <a class="anchor" id="head-1"></a>

For Efficientdet we use `coco based dataset`, you can refer to the original coco dataset for this tutorial

Coco based dataset should contain a folder "images" with all the images and a json file named annotations.json containing the annotations, and a label_map.txt file containing the class names (each class name in a line)

- Download coco based data and format it for API usage. Note that download, unzip and tar creating take some time depending on network speed and machine
- Create train and eval datasets
- Update information on train and eval datasets
- Upload data (201 indicates dataset was uploaded)
- Perform "dataset convert" to generate tfrecords

**If using custom dataset; it should follow this dataset structure**
```
DATA_DIR/train2017
├── annotations.json
├── images
    ├── image_name_1.jpg
    ├── image_name_2.jpg
    ├── ...

DATA_DIR/val2017
├── annotations.json
├── images
    ├── image_name_1.jpg
    ├── image_name_2.jpg
    ├── ...

```

In [ ]:
DATA_DIR = "efficientdet_data" #FIXME4 Set DATA_DIR to the path where the dataset is to be downloaded/or already present
os.environ['DATA_DIR']= DATA_DIR
!mkdir -p $DATA_DIR

In [ ]:
dataset_to_be_used = "default" #FIXME5 #default/custom; default for the dataset used in this tutorial notebook; custom for a different dataset

In [ ]:
if dataset_to_be_used == "default":
    !bash ../dataset_prepare/coco/download_coco.sh $DATA_DIR
    # Remove existing data
    !rm -rf $DATA_DIR/train2017/images
    !rm -rf $DATA_DIR/val2017/images
    # Rearrange data in the required format
    !mkdir -p $DATA_DIR/train2017/
    !mkdir -p $DATA_DIR/val2017/
    !mv $DATA_DIR/raw-data/train2017 $DATA_DIR/train2017/images
    !mv $DATA_DIR/raw-data/annotations/instances_train2017.json $DATA_DIR/train2017/annotations.json
    !mv $DATA_DIR/raw-data/val2017 $DATA_DIR/val2017/images
    !mv $DATA_DIR/raw-data/annotations/instances_val2017.json $DATA_DIR/val2017/annotations.json
    !cp ../dataset_prepare/coco/label_map.txt $DATA_DIR/train2017/
    !cp ../dataset_prepare/coco/label_map.txt $DATA_DIR/val2017/

In [ ]:
# Verify the downloaded dataset
!if [ ! -d $DATA_DIR/train2017/images ]; then echo 'Images folder not found'; else echo 'Found images folder';fi
!if [ ! -f $DATA_DIR/train2017/annotations.json ]; then echo 'annotations file not found'; else echo 'Found annotations file';fi
!if [ ! -d $DATA_DIR/val2017/images ]; then echo 'Images folder not found'; else echo 'Found images folder';fi
!if [ ! -f $DATA_DIR/val2017/annotations.json ]; then echo 'annotations file not found'; else echo 'Found annotations file';fi

In [ ]:
!tar -C $DATA_DIR/train2017/ -czf coco_train.tar.gz images annotations.json label_map.txt
!tar -C $DATA_DIR/val2017/ -czf coco_val.tar.gz images annotations.json label_map.txt

In [ ]:
train_dataset_path =  "coco_train.tar.gz"
eval_dataset_path = "coco_val.tar.gz"
test_dataset_path = "coco_val.tar.gz"

In [ ]:
# Create train dataset
ds_type = "object_detection"
ds_format = "coco"
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())
dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name":"Train dataset",
                       "description":"My train dataset with coco"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
files = [("file",open(train_dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

In [ ]:
# Create eval dataset
ds_type = "object_detection"
ds_format = "coco"
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())
eval_dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name":"Eval dataset",
                       "description":"My eval dataset with coco"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{eval_dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
files = [("file",open(eval_dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{eval_dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

In [ ]:
# Create testing dataset for inference
ds_type = "object_detection"
ds_format = "raw"
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data, headers=headers)

print(response)
print(response.json())
test_dataset_id = response.json()["id"]

In [ ]:
# Upload
files = [("file",open(test_dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{test_dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

### List the created datasets <a class="anchor" id="head-2"></a>

In [ ]:
endpoint = f"{base_url}/dataset"

response = requests.get(endpoint, headers=headers)

print(response)
# print(response.json()) ## Uncomment for verbose list output
print("id\t\t\t\t\t type\t\t\t format\t\t name")
for rsp in response.json():
    print(rsp["id"],"\t",rsp["type"],"\t",rsp["format"],"\t\t",rsp["name"])

### Dataset convert Action <a class="anchor" id="head-3"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/dataset/{dataset_id}/specs/convert_efficientdet/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes
specs["coco_config"]["num_shards"] = 256
specs["coco_config"]["tag"] = "train"

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/dataset/{dataset_id}/specs/convert_efficientdet"

response = requests.post(endpoint,data=data, headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = None
actions = ["convert_efficientdet"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/dataset/{dataset_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

ds_convert_id = response.json()[0]

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = ds_convert_id
endpoint = f"{base_url}/dataset/{dataset_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Now, repeat the same for the eval dataset
# Get default spec schema
endpoint = f"{base_url}/dataset/{eval_dataset_id}/specs/convert_efficientdet/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))


In [ ]:
# Apply changes
specs["coco_config"]["num_shards"] = 256
specs["coco_config"]["tag"] = "val"

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/dataset/{eval_dataset_id}/specs/convert_efficientdet"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = None
actions = ["convert_efficientdet"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/dataset/{eval_dataset_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

eval_ds_convert_id = response.json()[0]

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = eval_ds_convert_id
endpoint = f"{base_url}/dataset/{eval_dataset_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

### Create model <a class="anchor" id="head-4"></a>

In [ ]:
network_arch = "efficientdet"
encode_key = "tlt_encode"
data = json.dumps({"network_arch":network_arch,"encryption_key":encode_key})

endpoint = f"{base_url}/model"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())
model_id = response.json()["id"]

### List models <a class="anchor" id="head-5"></a>

In [ ]:
endpoint = f"{base_url}/model"

response = requests.get(endpoint, headers=headers)

print(response)
# print(response.json()) ## Uncomment for verbose list output
print("model id\t\t\t     network architecture")
for rsp in response.json():
    print(rsp["id"],rsp["network_arch"])

### Assign train, eval datasets <a class="anchor" id="head-6"></a>

- Note: make sure the order for train_datasets is [source ID, target ID]
- eval_dataset is kept same as target for demo purposes
- inference_dataset is kept as target for chaining with hifigan finetune

In [ ]:
dataset_information = {"train_datasets":[dataset_id],
                       "eval_dataset":eval_dataset_id,
                       "inference_dataset":test_dataset_id,
                       "calibration_dataset":dataset_id}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Assign PTM <a class="anchor" id="head-7"></a>

- Search for pretrained_efficientdet:efficientnet_b0 on NGC
- Assign it to the model

In [ ]:
 # Get pretrained model for efficientdet
model_list = f"{base_url}/model"
response = requests.get(model_list, headers=headers)

response_json = response.json()

# Search for ptm with given ngc path
ptm_id = None
for rsp in response_json:
    if  rsp["network_arch"] == network_arch and "pretrained_efficientdet:efficientnet_b0" in rsp["ngc_path"]:
        ptm_id = rsp["id"]
        print("Metadata for model with requested NGC Path")
        print(rsp)
        break
efficientdet_ptm = ptm_id

In [ ]:
ptm_information = {"ptm":efficientdet_ptm}
data = json.dumps(ptm_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Actions <a class="anchor" id="head-8"></a>

For all actions:
1. Get default spec schema and derive the default values
2. Modify defaults if needed
3. Post spec dictionary to the service
4. Run model action
5. Monitor job using retrieve
6. Download results using job download endpoint (if needed)

In [ ]:
job_map = {}

### Train <a class="anchor" id="head-9"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/train/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes to spec file as necessary
specs["training_config"]["num_epochs"] = 80
specs["training_config"]["train_batch_size"] = 8
specs["training_config"]["num_examples_per_epoch"] = 500 # Set this to number of images in dataset/num_gpu

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/train"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = None
actions = ["train"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["train"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['train']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents once the above job shows "Done" status
# Download output of efficientdet train (Note: will take time)
job_id = job_map["train"]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xvf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
model_downloaded_path = f"{workdir}/{job_id}"

In [ ]:
# Look for a model.tlt file
!ls {model_downloaded_path}/weights/

### Evaluate <a class="anchor" id="head-10"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/evaluate/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes to the specs if necessary

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/evaluate"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train"]
actions = ["evaluate"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["evaluate"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['evaluate']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

### Optimize <a class="anchor" id="head-11"></a>

- We optimize the trained model by pruning and retraining in the following cells

### Apply specs for prune <a class="anchor" id="head-12"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/prune/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes to specs if necessary like
specs["pruning_threshold"] = 0.7

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/prune"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

### Apply specs for retrain <a class="anchor" id="head-13"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/retrain/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes to specs as necessary
specs["training_config"]["num_epochs"] = 80
specs["training_config"]["train_batch_size"] = 8
specs["training_config"]["num_examples_per_epoch"] = 500 # Set this to number of images in dataset/num_gpu

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/retrain"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

### Run Actions <a class="anchor" id="head-14"></a>

We use the API's job chaining feature to prune, retrain and evaluate the retrained model

In [ ]:
# Run actions
parent = job_map["train"]
actions = ["prune","retrain","evaluate"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["prune"] = response.json()[0]
job_map["retrain"] = response.json()[1]
job_map["eval_retrain"] = response.json()[2]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell (prune)
job_id = job_map['prune']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Monitor job status by repeatedly running this cell (retrain)
job_id = job_map['retrain']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Monitor job status by repeatedly running this cell (evaluate)
job_id = job_map['eval_retrain']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Optional cancel job - for jobs that are pending/running (retrain)

# job_id = job_map['retrain']
# endpoint = f"{base_url}/model/{model_id}/job/{job_id}/cancel"

# response = requests.post(endpoint, headers=headers)

# print(response)
# print(response.json())

In [ ]:
# Optional delete job - for jobs that are error/done (retrain)

# job_id = job_map['retrain']
# endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

# response = requests.delete(endpoint, headers=headers)

# print(response)
# print(response.json())

### Export: FP32 <a class="anchor" id="head-15"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/export/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes
specs["data_type"] = "fp32"

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/export"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train"]
actions = ["export"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["export_fp32"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['export_fp32']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

### Export: INT8 <a class="anchor" id="head-16"></a>

- Note - if another export job is in Pending state for this model experiment, modifying specs would apply to the Pending experiment

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/export/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes
specs["data_type"] = "int8"
specs["batches"] = 10
specs["batch_size"] = 8
specs["max_batch_size"] = 1

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/export"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train"]
actions = ["export"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["export_int8"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['export_int8']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents once the above job shows "Done" status
job_id = job_map["export_int8"]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xvf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
model_downloaded_path = f"{workdir}/{job_id}"

In [ ]:
# Look for the generated .etlt file, tensorrt .engine file and cal.bin calibration cache file
!ls {model_downloaded_path}

### Model convert using TAO-Converter <a class="anchor" id="head-17"></a>

- Here, we use the INT8 exported model to convert to target platform

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/convert/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes
specs["t"] = "int8"
specs["b"] = 8
specs["p"] = "image_arrays:0,1x512x512x3,8x512x512x3,16x512x512x3"

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/convert"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["export_int8"]
actions = ["convert"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["model_convert"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['model_convert']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

### TAO inference <a class="anchor" id="head-18"></a>

- Run inference on a set of images using the .tlt model created at train step

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/inference/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes to specs if necessary

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/inference"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train"]
actions = ["inference"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["inference_tlt"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['inference_tlt']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents once the above job shows "Done" status
job_id = job_map["inference_tlt"]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xvf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
inference_out_path = f"{workdir}/{job_id}"

In [ ]:
# Inference output must be here
!ls {inference_out_path}/images_annotated

In [ ]:
from IPython.display import Image
import glob
sample_image = glob.glob(f"{inference_out_path}/images_annotated/*.jpg")[0]
Image(filename=sample_image) 